# Capítulo 06 — Fine-Tuning: A Especialização do Modelo

Neste capítulo, vamos transformar o nosso GPTMini em um especialista em detectar Spam. 

--- 
### 🎯 O Poder da Adaptação
O Fine-tuning pega o conhecimento estrutural do Backbone e o canaliza para uma tarefa de classificação binária.

![Pretrain vs Finetune](./infograficos/01-pretrain-vs-finetune.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        get_ipython().system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Especialista pronto em: {device}")

# 📂 Carregamento do Checkpoint do Cap 05
if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 Por favor, suba o 'gpt_checkpoint.pt' gerado no Capítulo 05:")
    uploaded = files.upload()

ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
stoi, itos = ckpt['stoi'], ckpt['itos']
encode = lambda s: [stoi[c] for c in s.lower() if c in stoi]
vocab_size = len(stoi)
print(f"🧠 Vocabulário de Caracteres Carregado: {vocab_size} tokens.")

## 1. Dataset Supervisionado

Usaremos frases curtas rotuladas como Spam (1) ou Normal (0).

In [ ]:
raw_data = [
    ("ganhe 1 milhao agora clique aqui", 1), # Spam
    ("oferta imperdivel premio gratis", 1),   # Spam
    ("seu premio esta esperando resgate", 1), # Spam
    ("ola tudo bem como voce esta", 0),      # Normal
    ("reuniao de equipe amanha as dez", 0),   # Normal
    ("voce vai no churrasco no domingo", 0)   # Normal
]

def build_clf_dataset(data, max_len=32):
    X, Y = [], []
    for text, label in data:
        ids = encode(text)
        ids = ids[:max_len] + [stoi.get(' ', 0)] * (max_len - len(ids))
        X.append(ids); Y.append(label)
    return torch.tensor(X).to(device), torch.tensor(Y).to(device)

X_train, Y_train = build_clf_dataset(raw_data)
print(f"📊 Dataset Processado: {len(X_train)} exemplos prontos.")

## 2. Classifier Head

Plugamos uma camada linear no topo do Backbone para tomar a decisão final.

![Head](./infograficos/02-classification-head.png)

In [ ]:
class GPTClassifier(nn.Module):
    def __init__(self, backbone, num_classes=2):
        super().__init__(); self.backbone = backbone
        self.clf_head = nn.Linear(backbone.config.d_model, num_classes)
    def forward(self, x):
        x = self.backbone.emb(x)
        x = self.backbone.blocks(x)
        x = self.backbone.ln_f(x)
        return self.clf_head(x[:, -1, :])

backbone = GPTMini(ckpt['config']).to(device)
backbone.load_state_dict(ckpt['state_dict'])

model = GPTClassifier(backbone).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
print("🏗️ Modelo especializado pronto.")

## 3. Treinamento e Avaliação

Observamos o modelo aprendendo a separar as classes Spam e Normal.

In [ ]:
loss_history = []
model.train()
for step in range(201):
    logits = model(X_train)
    loss = F.cross_entropy(logits, Y_train)
    optimizer.zero_grad(); loss.backward(); optimizer.step()
    loss_history.append(loss.item())
    if step % 50 == 0:
        acc = (torch.argmax(logits, dim=-1) == Y_train).float().mean()
        print(f"Passo {step:03d} | Erro: {loss.item():.4f} | Acurácia: {acc*100:.1f}%")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#34A853')
plt.title("Curva de Especialização")
plt.show()

## 🏁 Conclusão

Neste capítulo, você viu como transformar um cérebro digital genérico em uma ferramenta útil de segurança.

![Confusion Matrix](./infograficos/05-metricas-confusion-matrix.png)